In [71]:
%%configure -f
{"driverMemory": "4G", "driverCores": 4, "executorMemory": "12G", "executorCores": 4, "numExecutors": 3}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,,,None,✔


In [72]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy', 'seaborn', 'statsmodels', 'pyarrow'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, DataFrame

sc: SparkContext
spark: SparkSession

distance_calculation_df = (
    spark
    .read
    .format("mongodb")
    .option("database", "enhancer3d")
    .option("collection", "distance_calculation")
    .option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.PaginateIntoPartitionsPartitioner")
    .option("partitioner.options.max.number.of.partitions", 128)
    .load()
    .repartition(
        12,
        '_id.project_id',
        '_id.region_id',
        '_id.gene_id',
        '_id.enh_id'
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
links_gm12878_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/GM12878_EP_hg38_liftovered.parquet")
    .alias("links")
    .repartition(
        12,
        'gene_id',
        'enh_id'
    )
)

links_hffc6_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/HFFC6_EP_hg38_liftovered.parquet")
    .alias("links")
    .repartition(
        12,
        'gene_id',
        'enh_id'
    )
)

links_h1esc_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/H1ESC_EP_hg38_liftovered.parquet")
    .alias("links")
    .repartition(
        12,
        'gene_id',
        'enh_id'
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
%%pretty
links_gm12878_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

enh_id,gene_id,pval,qval
chr1:843600-844990,ENSG00000197049,1.180354,1.0
chr1:966620-966700,ENSG00000197049,0.642743,2.0
chr1:843600-844990,ENSG00000188976,1.016813,3.0
chr1:966620-966700,ENSG00000188976,2.58213,4.0
chr1:1013580-1013980,ENSG00000188976,2.073247,5.0


In [75]:
from pyspark.sql import functions as F, types as T
import numpy as np
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))

@F.udf(T.DoubleType())
def mannwhiteneyu(ref, mod):
    result = stats.mannwhitneyu(np.array(ref), np.array(mod), alternative='two-sided')
    return float(result.pvalue)

@F.udf(T.DoubleType())
def bonferroni_correction(pvalues, alpha=0.05):
    reject, pvals_corrected, _, _ = multipletests(pvalues, alpha=alpha, method='bonferroni')
    return float(np.mean(pvals_corrected))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
gm12878_neanderthal_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_GM12878')
        & (F.col('_id.ensemble_id').like('models3D_GM12878_Nean_models3D_GM12878_Nean_results%'))
        & (F.col('gene_type') == 'protein_coding')
        # & (F.col('enh_tSS_distance') < 20_000)
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        # 'dist',
        'avg_dist',
        'enh_tSS_distance'
    )
    # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("gm12878")
    .cache()
)

hffc6_neanderthal_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_HFFC6')
        & (F.col('_id.ensemble_id').like('models3D_HFFC6_Nean_models3D_HFFC6_Nean_results%'))
        & (F.col('gene_type') == 'protein_coding')
        # & (F.col('enh_tSS_distance') < 20_000)
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        # 'dist',
        'avg_dist',
        'enh_tSS_distance'
    )
     # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("hffc6")
    .cache()
)

h1esc_neanderthal_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_H1ESC')
        & (F.col('_id.ensemble_id').like('models3D_H1ESC_Nean_models3D_H1ESC_Nean_results%'))
        & (F.col('gene_type') == 'protein_coding')
        # & (F.col('enh_tSS_distance') < 20_000)
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        # 'dist',
        'avg_dist',
        'enh_tSS_distance'
    )
     # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("h1esc")
    .cache()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
gm12878_existing_links_df = (
    gm12878_neanderthal_df_ref
    .join(
        other=links_gm12878_df,
        on=F.expr("gm12878.gene_id = links.gene_id AND gm12878.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        gm12878_neanderthal_df_ref.region_id,
        gm12878_neanderthal_df_ref.gene_id,
        gm12878_neanderthal_df_ref.enh_id,
        gm12878_neanderthal_df_ref.avg_dist
    )
)

hffc6_existing_links_df = (
    hffc6_neanderthal_df_ref
    .join(
        other=links_hffc6_df,
        on=F.expr("hffc6.gene_id = links.gene_id AND hffc6.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        hffc6_neanderthal_df_ref.region_id,
        hffc6_neanderthal_df_ref.gene_id,
        hffc6_neanderthal_df_ref.enh_id,
        hffc6_neanderthal_df_ref.avg_dist
    )
)

h1esc_existing_links_df = (
    h1esc_neanderthal_df_ref
    .join(
        other=links_h1esc_df,
        on=F.expr("h1esc.gene_id = links.gene_id AND h1esc.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        h1esc_neanderthal_df_ref.region_id,
        h1esc_neanderthal_df_ref.gene_id,
        h1esc_neanderthal_df_ref.enh_id,
        h1esc_neanderthal_df_ref.avg_dist
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# gm12878_hffc6_common_links_df = (
#     gm12878_existing_links_df
#     .join(
#         other=hffc6_existing_links_df,
#         on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
#         how="inner"
#     )
#     .select(
#         gm12878_existing_links_df.region_id,
#         gm12878_existing_links_df.gene_id,
#         gm12878_existing_links_df.enh_id,
#         gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist'),
#         hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
#     )
# )
#
# gm12878_hffc6_gm12878_only_links_df = (
#     gm12878_existing_links_df
#     .join(
#         other=hffc6_existing_links_df,
#         on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
#         how="left_anti"
#     )
#     .select(
#         gm12878_existing_links_df.region_id,
#         gm12878_existing_links_df.gene_id,
#         gm12878_existing_links_df.enh_id,
#         gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist')
#     )
# )
#
# gm12878_hffc6_hffc6_only_links_df = (
#     hffc6_existing_links_df
#     .join(
#         other=gm12878_existing_links_df,
#         on=F.expr("hffc6.gene_id = gm12878.gene_id AND hffc6.enh_id = gm12878.enh_id"),
#         how="left_anti"
#     )
#     .select(
#         hffc6_existing_links_df.region_id,
#         hffc6_existing_links_df.gene_id,
#         hffc6_existing_links_df.enh_id,
#         hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
#     )
# )

In [58]:
import os

# write all to csv into /work/playground/links/experiment_3
os.makedirs("/work/playground/links/experiment_3", exist_ok=True)

(gm12878_neanderthal_df_ref.repartition(12)
 .toPandas().to_parquet("/work/playground/links/experiment_3/gm12878_neanderthal_df_ref.parquet", index=False))
# hffc6_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_3/hffc6_neanderthal_df_ref.parquet", index=False)
# h1esc_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_3/h1esc_neanderthal_df_ref.parquet", index=False)
#
# gm12878_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_3/gm12878_existing_links.csv", index=False)
# hffc6_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_3/hffc6_existing_links.csv", index=False)
# h1esc_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_3/h1esc_existing_links.csv", index=False)

# gm12878_hffc6_common_links_df.toPandas().to_csv("/work/playground/links/experiment_3/gm12878_hffc6_common_links.csv", index=False)

# gm12878_hffc6_gm12878_only_links_df.toPandas().to_csv("/work/playground/links/experiment_3/gm12878_hffc6_gm12878_only_links.csv", index=False)
# gm12878_hffc6_hffc6_only_links_df.toPandas().to_csv("/work/playground/links/experiment_3/gm12878_hffc6_hffc6_only_links.csv", index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user
